In [ ]:
# @title
import pickle
import pandas as pd
import glob
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [ ]:
experiment_name = "2024.01.04-03_T5_xlarge_zero-shot_NEW"

In [ ]:
# @title
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"

output_data_path = base_path + "/../Flan-T5/data/output/" + experiment_name
log_filename = base_path + "/../Flan-T5/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# @title
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
# model_id = "google/flan-t5-small" # 60M
# model_id = "google/flan-t5-base" # 220M
# model_id = "google/flan-t5-large" # 800M
model_id = "google/flan-t5-xl" # 3B
# model_id = "google/flan-t5-xxl" # 11B

In [ ]:
# @title
# nbit = 4
# # llamasize = '7B'
# llamasize = '13B'
# temp = 0.1
# outdir = 'data/output/2023.11.13-1_T5_run_completo-large___TEST/'
# outdir = 'data/output/2023.11.22-6_T5_run_completo-base_zero-shot-REBOOT/'
# outfilename = outdir+f'sdg_classification_T5_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
outfilename = output_data_path+f'/sdg_classification_T5_1020-texts.xlsx'

outfilename

'/content/gdrive/MyDrive/Colab/SDG/data/../Flan-T5/data/output/2024.01.04-03_T5_xlarge_zero-shot_NEW/sdg_classification_T5_1020-texts.xlsx'

In [ ]:
data = pd.read_excel(input_data_path)

In [ ]:
# @title
data_sel = data
# data_sel = data.sample(10)

In [ ]:
data_sel = data

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)
    if res[3] == "<pad> SDG_99</s>":
      res = (res[0], res[1], res[2], "<pad> SDG_0</s>")
    results.append(res)

len(results)

1020

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'id', 'true_label', 'predicted_label'])
results_processed

,cnt,id,true_label,predicted_label
0,21,3889,0,<pad> SDG_0</s>
1,22,6576,0,<pad> SDG_0</s>
2,23,3912,0,<pad> SDG_0</s>
3,24,1565,0,<pad> SDG_0</s>
4,25,1033,0,<pad> SDG_0</s>
...,...,...,...,...
1015,16,2166,0,<pad> SDG_0</s>
1016,17,2512,0,<pad> SDG_16</s>
1017,18,2191,0,<pad> SDG_0</s>
1018,19,6073,0,<pad> SDG_0</s>


In [ ]:
# results_processed['label'] = results_processed['predicted_label'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['predicted_label'].str.replace('<pad> ', '')
results_processed['label'] = results_processed['label'].str.replace('</s>', '')

In [ ]:
results_processed.groupby('label').count()

,cnt,id,true_label,predicted_label
label,,,,
SDG_0,440,440,440,440
SDG_1,46,46,46,46
SDG_10,22,22,22,22
SDG_11,15,15,15,15
SDG_12,8,8,8,8
SDG_13,62,62,62,62
SDG_14,53,53,53,53
SDG_15,44,44,44,44
SDG_16,27,27,27,27


In [ ]:
results_processed['label'] = results_processed['label'].str.replace("_", " ")
results_processed['label'] = results_processed['label'].str.extract(r'SDG (\d+)')

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='id')
final.head(3)

,Unnamed: 0.1,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,label
0,0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,<pad> SDG_0</s>,0
1,1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,<pad> SDG_0</s>,0
2,2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,<pad> SDG_0</s>,0


In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

[]

In [ ]:
# final.to_excel(outfilename)

In [ ]:
final = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.12      0.88      0.21        60
           1       0.74      0.57      0.64        60
           2       1.00      0.57      0.72        60
           3       0.88      0.47      0.61        60
           4       0.90      0.58      0.71        60
           5       0.95      0.70      0.81        60
           6       1.00      0.45      0.62        60
           7       0.93      0.62      0.74        60
           8       0.49      0.60      0.54        60
           9       0.71      0.17      0.27        60
          10       0.82      0.30      0.44        60
          11       0.93      0.23      0.37        60
          12       0.88      0.12      0.21        60
          13       0.77      0.80      0.79        60
          14       0.94      0.83      0.88        60
          15       0.86      0.63      0.73        60
          16       0.81      0.37      0.51        60

    accuracy    

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

In [ ]:
report_df

,precision,recall,f1-score,support
0,12.045455,88.333333,21.200000,60.000000
1,73.913043,56.666667,64.150943,60.000000
2,100.000000,56.666667,72.340426,60.000000
3,87.500000,46.666667,60.869565,60.000000
4,89.743590,58.333333,70.707071,60.000000
5,95.454545,70.000000,80.769231,60.000000
6,100.000000,45.000000,62.068966,60.000000
7,92.500000,61.666667,74.000000,60.000000
8,49.315068,60.000000,54.135338,60.000000
9,71.428571,16.666667,27.027027,60.000000


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

80.83269903649558 52.25490196078431 52.254901960784316 57.6428417468125


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

80.8 52.3 52.3 57.6
